In [1]:
import pandas as pd
import numpy as np
import math


In [ ]:
model

In [3]:
path = '~/Documents/Python/2018/FM/1_ContentBase/'

In [4]:
all_ratings=pd.read_csv(path+"ratings.csv")

In [5]:
import pandas as pd 
def rstr(df):
    print('/n'' Structure of Data:','\n''Rows X Columns: ',df.shape,'\n'
        '\n''Features and value:''\n',df.apply(lambda x:[x.unique()]))
rstr(all_ratings)

('/n Structure of Data:', '\nRows X Columns: ', (264505, 4), '\n\nFeatures and value:\n', userId       [[12882.0, 320.0, 121987.0, 62046.0, 18127.0, ...
movieId      [[1.0, 32.0, 47.0, 50.0, 110.0, 150.0, 158.0, ...
rating       [[4.0, 3.5, 5.0, 4.5, 2.0, 2.5, 3.0, 0.5, 1.5,...
timestamp    [[1147195252.0, 1147195307.0, 1147195343.0, 11...
dtype: object)


In [6]:
all_ratings.head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [7]:
#transform data to transection
all_ratings["Favorable"] = all_ratings["rating"] > 3
#two_ratings=all_ratings
two_ratings=all_ratings[all_ratings['userId'].isin(range(3000))]
two_ratings["Datetime"] = pd.to_datetime(two_ratings['timestamp'], unit='s')
favorable_ratings = two_ratings[two_ratings["Favorable"]]
favorable_reviews_by_users = dict((k, frozenset(v.values)) 
    for k, v in favorable_ratings.groupby("userId")["movieId"])
num_favorable_by_movie = two_ratings[["movieId", "Favorable"]].groupby("movieId").sum()     

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#two_ratings.userId.unique()

In [9]:
frequent_itemsets = {}
# more then 30% is the popular movie
min_num_req=len(two_ratings.userId.unique())*0.6
frequent_itemsets[1]=dict((frozenset((movie_id,)),row["Favorable"])
    for movie_id, row in num_favorable_by_movie.iterrows()
    if row["Favorable"] > min_num_req)
print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_num_req))
#There are 16 movies with more than 50 favorable reviews

There are 25 movies with more than 12.0 favorable reviews


In [10]:
#frequent_itemsets[1]

In [11]:
#frequent_itemsets

In [12]:
from collections import defaultdict
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_num_req):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_num_req])
 

In [13]:
for k in range(2, 4):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],min_num_req)
    if len(cur_frequent_itemsets)==0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        frequent_itemsets[k] = cur_frequent_itemsets

I found 324 frequent itemsets of length 2
I found 2915 frequent itemsets of length 3


In [14]:
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))
#There are 15285 candidate rules
print(candidate_rules[:4])

There are 9418 candidate rules
[(frozenset([]), 589), (frozenset([]), 1196), (frozenset([]), 593), (frozenset([]), 4306)]


In [22]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {
    candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() 
if confidence > 0.8}
print(len(rule_confidence))

4630


In [25]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)
for index in range(3):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}\n".format(rule_confidence[(premise, conclusion)]))

Rule #1
Rule: If a person recommends frozenset([6539, 1214]) they will also recommend 356
 - Confidence: 1.000

Rule #2
Rule: If a person recommends frozenset([4306, 527]) they will also recommend 7153
 - Confidence: 1.000

Rule #3
Rule: If a person recommends frozenset([457, 3578]) they will also recommend 5952
 - Confidence: 1.000



In [42]:
movie_name_data.columns

Index([u'movieId', u'title', u'genres'], dtype='object')

In [41]:
sorted_confidence[0][0]

(frozenset({1214, 6539}), 356)

In [43]:
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["movieId"] == movie_id]["title"]
    title = title_object.values[0]
    return title
movie_name_data = pd.read_csv(path+"movies.csv")
#movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "", "Action", "Adventure",
#                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
#                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

#print(movie_name_data.head())
##SKIPPED
 
for index in range(3):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}\n".format(rule_confidence[(premise, conclusion)]))


Rule #1
Rule: If a person recommends Pirates of the Caribbean: The Curse of the Black Pearl (2003), Alien (1979) they will also recommend Forrest Gump (1994)
 - Confidence: 1.000

Rule #2
Rule: If a person recommends Shrek (2001), Schindler's List (1993) they will also recommend Lord of the Rings: The Return of the King, The (2003)
 - Confidence: 1.000

Rule #3
Rule: If a person recommends Fugitive, The (1993), Gladiator (2000) they will also recommend Lord of the Rings: The Two Towers, The (2002)
 - Confidence: 1.000



In [44]:
test_dataset = all_ratings[~all_ratings['userId'].isin(range(200))]
test_favorable = test_dataset[test_dataset["Favorable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) 
    for k, v in test_favorable.groupby("userId")["movieId"])
 
###7.2 Count the correct instances where the premise leads to the conclusion
correct_counts=defaultdict(int)
incorrect_counts=defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise,conclusion=candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule]+=1
            else:
                incorrect_counts[candidate_rule]+=1
test_confidence = {candidate_rule:
(correct_counts[candidate_rule] /
    float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule]))
for candidate_rule in rule_confidence}
print(len(test_confidence))

4630


In [45]:
for index in range(3):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}"
        .format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}\n".format(test_confidence.get((premise, conclusion), -1)))
 

Rule #1
Rule: If a person recommends Pirates of the Caribbean: The Curse of the Black Pearl (2003), Alien (1979) they will also recommend Forrest Gump (1994)
 - Train Confidence: 1.000
 - Test Confidence: 0.784

Rule #2
Rule: If a person recommends Shrek (2001), Schindler's List (1993) they will also recommend Lord of the Rings: The Return of the King, The (2003)
 - Train Confidence: 1.000
 - Test Confidence: 0.806

Rule #3
Rule: If a person recommends Fugitive, The (1993), Gladiator (2000) they will also recommend Lord of the Rings: The Two Towers, The (2002)
 - Train Confidence: 1.000
 - Test Confidence: 0.835

